# 信号塔覆盖

## 目的和先决条件

在此示例中，我们将解决一个简单的覆盖问题：如何构建一个信号塔网络以提供信号覆盖给尽可能多的人。我们将构建业务问题的数学模型，然后用 Gurobi Python API 进行建模，并计算最佳解决方案。

该建模示例处于初级阶段，我们假设你了解Python，并且具有一些有关构建数学优化模型的知识。

**注意：** 你可以通过单击 [此处](https://github.com/arvinxx/gurobi-and-mathematical-modeling/archive/master.zip) 下载包含此示例和其他示例的代码。为了正确运行此 Jupyter Notebook，你必须具有 Gurobi 许可证。如果你没有，则可以**商业用户身份**申请 [试用许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或以**学术用户身份**下载 [免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses)。

## 动机

在过去的十年里，智能手机彻底改变了我们的生活，远远超出了我们的沟通方式。除了打电话,发短信,发邮件,现在全世界有超过二十亿人使用这些设备导航到书出租车,比较产品评论和价格,遵循新闻,看电影,听音乐,玩游戏,拍照,参与社交媒体,和许多其他应用程序。

蜂窝网络是一种手持式智能手机网络，其中每部手机通过蜂窝基站(蜂窝基站)的本地天线通过无线电波与电话网络进行通信。一个重要的问题是如何安置手机信号塔，以便向最多的人提供信号覆盖。

## 问题描述

一家电信公司需要建造一组蜂窝塔，以为给定城市的居民提供信号覆盖范围。已经确定了可以建造塔楼的许多潜在位置。这些塔有固定的范围，并且-由于预算限制，只能建造数量有限的塔。鉴于这些限制，该公司希望将覆盖范围扩大到尽可能多的人口。为了简化问题，该公司将其希望覆盖的区域划分为一组区域，每个区域都有已知的人口。然后，目标是选择公司应在哪些潜在位置上建造手机信号塔，以便覆盖尽可能多的人。

蜂窝基站覆盖问题是最大覆盖位置问题[1]的一个实例。它也与集合覆盖问题（Set Cover Problem）有关。集合覆盖问题出现在许多不同的领域，非常重要的应用来自航空工业。例如，机组调度和航道分配问题[2]。

## 解决方法

数学优化（也称为数学编程）是一种声明性方法，其中建模者制定了一个优化问题，该问题捕获了复杂决策问题的关键特征。然后 Gurobi 使用最先进的数学和计算机科学技术来解决这一类数学优化问题。


数学优化模型具有五个组成部分：

* 数据集（Sets）
* 参数(Parameters)
* 决策变量(Decision variables)
* 约束（Constraints）
* 目标函数（Objective function(s)）

我们现在提出一个混合整数规划（MIP）模型来解决信号塔覆盖问题。

## Model Formulation

### Sets and Indices

$i \in T$: Index and set of potential sites to build a tower.

$j \in R$: Index and set of regions.

$G(T,R,E)$: A bipartite graph defined over the set $T$ of potential sites to build a tower,  the set of regions $R$ that we want to cover, and $E$ is the set of edges, where we have an edge $(i,j) \in E$ if region $j \in R$ can be covered by a tower on location $i \in T$.

### Parameters

$c_{i} \in \mathbb{R}^+$: The cost of setting up a tower at site $i$.

$p_{j} \in \mathbb{N}$: The population at region $j$.

### Decision Variables

$covered_{j} \in \{0, 1 \}$: This variable is equal to 1 if region $j$ is covered; and 0 otherwise.

$build_{i} \in \{0, 1 \}$: This variable is equal to 1 if tower $i$ is built; and 0 otherwise.

### Objective Function(s)

- **Population covered**. We seek to maximize the total population covered by the towers.

\begin{equation}
\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### Constraints

- **Coverage**. For each region $j \in R$ ensure that at least one tower that covers a region must be selected.

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **Budget**. We need to ensure that the total cost of building towers do not exceed the allocated budget.

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

## Python Implementation

This example considers a bipartite graph for 6 towers and 9 regions. The following table illustrates which regions (columns) are covered by each cell tower site (rows).

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| Tower 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| Tower 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| Tower 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| Tower 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| Tower 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| Tower 5 | - | - | - | 1 | 1 | - | - | - |  1 |

The population at each region is stated in the following table.

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Population | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

The cost to build a cell tower at each location site is stated inthe following table.

| <i></i> | Cost (millions of USD) |
| --- | --- |
| Tower 0 | 4.2 |
| Tower 1 | 6.1 |
| Tower 2 | 5.2 |
| Tower 3 | 5.5 |
| Tower 4 | 4.8 |
| Tower 5 | 9.2 | 

The allocated budget is $\$20,000,000$.

We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.


In [1]:
import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.0.0 and Python 3.7.0

# Parameters
budget = 20
regions, population = gp.multidict({
    0: 523, 1: 690, 2: 420,
    3: 1010, 4: 1200, 5: 850,
    6: 400, 7: 1008, 8: 950
})

sites, coverage, cost = gp.multidict({
    0: [{0,1,5}, 4.2],
    1: [{0,7,8}, 6.1],
    2: [{2,3,4,6}, 5.2],
    3: [{2,5,6}, 5.5],
    4: [{0,2,6,7,8}, 4.8],
    5: [{3,4,8}, 9.2]
})


### Model Deployment

We now determine the model for the Cell Tower Coverage Problem, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to build towers that maximizes  the coverage of the population given the budget allocated.

In [2]:
# MIP  model formulation
m = gp.Model("cell_tower")

build = m.addVars(len(sites), vtype=GRB.BINARY, name="Build")
is_covered = m.addVars(len(regions), vtype=GRB.BINARY, name="Is_covered")

m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
m.addConstr(build.prod(cost) <= budget, name="budget")

m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

m.optimize() 

Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xfa0fabb2
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 21 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 7.051000e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7051.0000000 7051.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.02 second

## 分析

优化模型的结果表明，$\$20,000,000$ 预算可以覆盖的最大人数为 7,051 人。让我们看看实现最佳结果的解决方案。

### Cell Tower Build Plan

This plan determines at which site locations to build a cell tower.

In [3]:
# display optimal values of decision variables

for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Build a cell tower at location Tower {tower}.")


 Build a cell tower at location Tower 0.

 Build a cell tower at location Tower 2.

 Build a cell tower at location Tower 4.


### Demand Fulfillment Metrics

- **Coverage**: Percentage of the population covered by the cell towers built.

In [4]:
# Percentage of the population covered by the cell towers built is computed as follows.

total_population = 0

for region in range(len(regions)):
    total_population += population[region]

coverage = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the cell towers build plan is: {coverage} %")


 The population coverage associated to the cell towers build plan is: 100.0 %


### Resources Utilization Metrics

- **Budget consumption**: Percentage of the budget allocated to build the cell towers.

In [5]:
# Percentage of budget consumed to build cell towers

total_cost = 0

for tower in range(len(sites)):
    if (abs(build[tower].x) > 0.5):
        total_cost += cost[tower]*int(build[tower].x)

budget_consumption = round(100*total_cost/budget, 2)

print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")


 The percentage of budget consumed associated to the cell towers build plan is: 71.0 %


##  Conclusion

In this example, we address the problem of building cell towers to provide signal coverage  to the largest number of people while satisfying a budget constraint. We learned how to formulate the problem as a MIP model. Also, we learned how to implement the MIP model formulation and solve it using the Gurobi Python API.

## References

[1] Richard Church and Charles R. Velle. "The Maximal Covering Location Problem". Papers in Regional Science, 1974, vol. 32, issue 1, 101-118.

[2] Tail Assignment Problem. https://www.gurobi.com/case_study/air-france-tail-assignment-optimization/

Copyright © 2020 Gurobi Optimization, LLC

翻译 By Arvin Xu